In [ ]:
!pip install kaggle


In [ ]:
from google.colab import files

# Upload the Kaggle API key (kaggle.json) that you downloaded
uploaded = files.upload()


Saving kaggle.json to kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d tourist55/alzheimers-dataset-4-class-of-images


alzheimers-dataset-4-class-of-images.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/alzheimers-dataset-4-class-of-images.zip

Archive:  /content/alzheimers-dataset-4-class-of-images.zip
replace Alzheimer_s Dataset/test/MildDemented/26 (19).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!pip install tensorflow_addons

In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, GlobalAveragePooling2D

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


TensorFlow Version: 2.14.0


**DATA AUGMENTATION**

In [ ]:
#base_dir = "/content/dataset"
base_dir = "/content/Alzheimer_s Dataset/"
root_dir = "./"
test_dir = base_dir + "test/"
train_dir = base_dir + "train/"
work_dir = root_dir + "dataset/"

if os.path.exists(work_dir):
    remove_tree(work_dir)


os.mkdir(work_dir)
copy_tree(train_dir, work_dir)
copy_tree(test_dir, work_dir)
print("Working Directory Contents:", os.listdir(work_dir))

Working Directory Contents: ['ModerateDemented', 'VeryMildDemented', 'NonDemented', 'MildDemented']


In [ ]:
CLASSES = [ 'NonDemented',
            'VeryMildDemented',
            'MildDemented',
            'ModerateDemented']

IMG_SIZE = 150
IMAGE_SIZE = [150, 150]
DIM = (IMG_SIZE, IMG_SIZE)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
ZOOM = [.99, 1.01]
BRIGHT_RANGE = [0.8, 1.2]
HORZ_FLIP = True
FILL_MODE = "constant"
DATA_FORMAT = "channels_last"
# Define the ImageDataGenerator with augmentation parameters
image_data_generator = ImageDataGenerator(
rescale=1.0 / 255,
brightness_range=BRIGHT_RANGE,
zoom_range=ZOOM,
data_format=DATA_FORMAT,
fill_mode=FILL_MODE,
horizontal_flip=HORZ_FLIP
)
# Create a generator for the training data from the working directory
train_data_gen = image_data_generator.flow_from_directory(
directory=work_dir,
target_size=IMAGE_SIZE,
batch_size=6500
                                                           ,
shuffle=False
)


Found 6400 images belonging to 4 classes.


In [ ]:
#Retrieving the data from the ImageDataGenerator iterator

train_data, train_labels = train_data_gen.next()

In [ ]:
#Getting to know the dimensions of our dataset

print(train_data.shape, train_labels.shape)

(6400, 150, 150, 3) (6400, 4)


In [ ]:
#Performing over-sampling of the data, since the classes are imbalanced

sm = SMOTE(random_state=42)

train_data, train_labels = sm.fit_resample(train_data.reshape(-1, IMG_SIZE * IMG_SIZE * 3), train_labels)

train_data = train_data.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

print(train_data.shape, train_labels.shape)


(12800, 150, 150, 3) (12800, 4)


**Deep Net**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import DenseNet121, VGG16

def custom_densenet(input_tensor):
    base_model = DenseNet121(include_top=False, weights='imagenet', input_tensor=input_tensor)
    return base_model

def custom_vgg16(input_tensor):
    base_model = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)
    return base_model

def deepnet():
    input_layer = Input(shape=(150, 150, 3))

    custom_densenet_model = custom_densenet(input_layer)
    custom_vgg16_model = custom_vgg16(input_layer)

    features_densenet = GlobalAveragePooling2D()(custom_densenet_model.output)
    features_vgg16 = GlobalAveragePooling2D()(custom_vgg16_model.output)

    features = concatenate([features_densenet, features_vgg16])

    x = Dense(256, activation='relu')(features)
    x = Dense(4, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=x)

    return model

model = deepnet()
model.summary()


In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc')
]

In [ ]:
#Splitting the data into train, test, and validation sets
train_data, test_data, train_labels, test_labels = train_test_split(train_data, train_labels, test_size = 0.2, random_state=42)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Fit the model
history = model.fit(train_data, train_labels, epochs=20, batch_size=8, validation_data=(test_data, test_labels))

Epoch 1/20
1280/1280 [==============================] - 275s 154ms/step - loss: 0.8133 - accuracy: 0.6357 - val_loss: 2.3239 - val_accuracy: 0.3715
Epoch 2/20
1280/1280 [==============================] - 189s 147ms/step - loss: 0.6421 - accuracy: 0.7015 - val_loss: 1.7905 - val_accuracy: 0.3289
Epoch 3/20
1280/1280 [==============================] - 197s 154ms/step - loss: 0.5431 - accuracy: 0.7539 - val_loss: 0.6458 - val_accuracy: 0.6926
Epoch 4/20
1280/1280 [==============================] - 190s 148ms/step - loss: 0.4730 - accuracy: 0.7879 - val_loss: 0.7367 - val_accuracy: 0.7277
Epoch 5/20
1280/1280 [==============================] - 189s 147ms/step - loss: 0.4130 - accuracy: 0.8229 - val_loss: 0.5267 - val_accuracy: 0.7883
Epoch 6/20
1280/1280 [==============================] - 188s 147ms/step - loss: 0.3370 - accuracy: 0.8569 - val_loss: 0.3473 - val_accuracy: 0.8605
Epoch 7/20
1280/1280 [==============================] - 189s 147ms/step - loss: 0.2734 - accuracy: 0.8911 - val_

In [ ]:
#Evaluating the model on the data
test_scores = model.evaluate(test_data, test_labels)
tprint("Testing Accuracy: %.2f%%"%(test_scores[1] * 100))